## 0. Pré-Processamento

### 0.1 Importando a(s) biblioteca(s) e o dataset

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('scrapping_roupas_marketplace.csv')

## 1. Limpeza da base

In [3]:
#Removendo a coluna indesejada
df = df.drop('Unnamed: 0', axis=1)

### 1.1 Tratando os nulos

Os nulos na categoria de nota não foram tratados e o trabalho para ajuste deve ficar a cargo de quem trabalhar com a base.

In [4]:
df.isna().sum()

Título             0
Reais              0
Centavos         243
Condição           0
Nota             648
N_Avaliações     648
Desconto        1333
Marca             23
Material         875
Gênero            87
Temporada       1072
Review1          692
Review2          853
Review3          974
dtype: int64

In [5]:
df[['Centavos', 'N_Avaliações', 'Desconto']] = df[['Centavos', 'N_Avaliações', 'Desconto']].fillna(0)
df[['Marca', 'Material', 'Gênero', 'Temporada', 'Review1', 'Review2', 'Review3']] =df[['Marca', 'Material', 'Gênero', 'Temporada', 'Review1', 'Review2', 'Review3']].fillna('N/A')

In [6]:
df[['Reais', 'Centavos']] = df[['Reais', 'Centavos']].astype(int).astype(str)
df['Preco'] = (df['Reais'] + '.' + df['Centavos']).astype(float)

In [7]:
df = df.drop(['Reais', 'Centavos'], axis=1)

### 1.1 Arrumando colunas

In [8]:
#Ajeitando as colunas do número de avaliações, tirando os ()
df['N_Avaliações'] = df['N_Avaliações'].apply(lambda x: int(x) if x == 0 else int(x[1:-1]))

In [9]:
#Arrumando as colunas de desconto, transformando em número
df['Desconto'] = df['Desconto'].apply(lambda x: int(str(x).replace('% OFF', ''))/100)

In [10]:
#Utilizando a coluna de "Condição" para identificar o número de itens vendidos em cada anúncio
dic_condicoes = {'Novo': 0, 
                 '1 vendido': 1,
                 '2': 2,
                 '3': 3,
                 '4': 4,
                 '+5': 5,
                 '+25': 25,
                 '+50': 50,
                 '+100': 100,
                 '+500': 500,
                 '+1000': 1000,
                 '+5mil': 5000,
                 '+10mil': 10000,
                 '+50mil': 50000,
                 }

df['Condição'] = df['Condição'].apply(lambda x: x.replace('Novo  |  ', ''))
df['Condição'] = df['Condição'].apply(lambda x: x.replace(' vendidos', ''))

df['n_vendidos_categoria'] = df['Condição'].replace(dic_condicoes)

In [11]:
df = df.drop('Condição', axis=1)

In [12]:
#Criando uma coluna com o preço original do item, antes do desconto
df['Preco_original'] = round(df['Preco']*(df['Desconto']+1), 2)

In [13]:
#Reordenando as colunas do dataframe
df = df[['Título', 'Preco_original', 'Desconto', 'Preco', 'n_vendidos_categoria',  'Marca', 'Material', 'Gênero', 'Temporada', 'Nota', 'N_Avaliações', 'Review1', 'Review2', 'Review3']]

In [14]:
#Renomeando algumas colunas para melhor entendimento
df = df.rename(columns={'Preco':'Preco_com_desconto', 'Desconto':'Desconto_percentual'})

In [15]:
df

,Título,Preco_original,Desconto_percentual,Preco_com_desconto,n_vendidos_categoria,Marca,Material,Gênero,Temporada,Nota,N_Avaliações,Review1,Review2,Review3
0,Pijama Cirurgico Scrub Roupa Privativa Hospit...,69.90,0.00,69.90,1000,TORONTO UNIFORMES PROFISSIONAIS,Poliéster,Sem gênero,N/A,4.0,273,"Preço justo , bem feito acabamento ok. Única s...","Tamanho bom, costuras de qualidade, porém teci...","Muito bom, só poderia ter os cortes na lateral..."
1,Kit 10 Cuecas Boxer Lupo Cueca Box Algodão Mas...,274.81,0.18,232.89,10000,Lupo,Algodão,Masculino,Outono/Inverno,4.5,3034,As cuecas são boas; porém você percebe na cost...,"Pelo preço promocional, com ctz tem algum defe...","Serviram muito bem, porém é nítido que o pano ..."
2,Kit Com 10 Cuecas Boxer Algodão Sem Costura Zo...,239.88,0.20,199.90,50000,Zorba,Algodão,Masculino,N/A,4.7,5682,O tecido é bom e são confortáveis. Só que a nu...,Vendo comentários de outros consumidores vejo ...,"Produto não é nem de longe original, tecido gr..."
3,Kit 10 Cuecas Boxer Mash Algodão Cotton Box Or...,279.86,0.22,229.39,10000,Mash,Algodão,Masculino,Primavera/Verão,4.6,1700,"As cuecas são boas, porém meu marido usa g e p...","E o tamanho certo, mas em baixo dela, fica mui...",Produto não parece.\nOriginal e veio 2 modelos...
4,Kit 3 Short Jeans Feminino Cintura Alta Barato...,140.83,0.09,129.20,1000,Menina Linda,Jean,Feminino,Primavera/Verão,4.4,507,Estou encantada com essas peças!.\nOs shorts s...,"Recomendo, tecido confortável, igual a foto.","Gostei muito dos shorts, material bom, para us..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2202,Bermuda Masculina Tática Thunder Poly Ripstop ...,131.78,0.10,119.80,5,Use Tático,Poliéster,Masculino,Primavera/Verão,4.3,3,Muito bom produto gostei muito.,Tá ótimo recomendo a todos.,N/A
2203,Camiseta Preta Juvenil Infantil Menina Menino ...,23.16,0.13,20.50,25,DestaQ,Algodão,Sem gênero infantil,N/A,NaN,0,N/A,N/A,N/A
2204,Blusa Personalizada Jaqueta Com Sua Marca Logo...,93.99,0.00,93.99,1,Inoffensive,N/A,Feminino,N/A,NaN,0,N/A,N/A,N/A
2205,Roupa Infantil Social Menino - Bebê Batizado F...,97.30,0.00,97.30,50,Balão Magico Kids,N/A,Bebês,N/A,4.7,14,"O roupa em si é linda, só a camisa que o tecid...",Amei.,Ficou perfeito no meu filho.


## 2. Salvando a base

In [16]:
df.to_csv('dataset_roupas_marketplacebr_limpo.csv')